<a href="https://colab.research.google.com/github/JoeOlang/NLP/blob/main/Pytorch/Classifying_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classifying Names With a Character Level RNN (PyTorch)

In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/Basics/RNN Name Classification

/content/drive/MyDrive/Colab Notebooks/NLP/Basics/RNN Name Classification


In [2]:
%ls

'Classifying Names.ipynb'   eng-fra.txt   names/


In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

In [5]:
def findfiles(path): return glob.glob(path)

print(findfiles('names/*.txt'))

['names/French.txt', 'names/Dutch.txt', 'names/Italian.txt', 'names/Vietnamese.txt', 'names/Korean.txt', 'names/Portuguese.txt', 'names/German.txt', 'names/Chinese.txt', 'names/English.txt', 'names/Japanese.txt', 'names/Greek.txt', 'names/Polish.txt', 'names/Russian.txt', 'names/Irish.txt', 'names/Czech.txt', 'names/Scottish.txt', 'names/Arabic.txt', 'names/Spanish.txt']


In [12]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Unicode to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build a cateogry lines dict, a list of names per lang.
category_lines = {}
all_categories =[]

# read file and split into lines
def readlines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findfiles('names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readlines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)



Slusarski


In [13]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [16]:
import torch

# find letter index from all_letters >> ONE-HOT-ENC
def letterToIndex(letter):
    return all_letters.find(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(letterToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([1, 57])


In [18]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [19]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [26]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.8355, -2.8521, -2.9204, -2.8689, -2.9517, -3.0252, -2.8649, -2.8294,
         -2.8673, -2.8910, -2.8454, -2.8438, -3.0373, -2.9247, -2.8410, -2.9046,
         -2.9252, -2.8307]], grad_fn=<LogSoftmaxBackward>)


In [28]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Chinese', 7)


In [33]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype = torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor


for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category = ', category, '/line =', line)

category =  Polish /line = Salomon
category =  English /line = Carmichael
category =  Irish /line = Clark
category =  Russian /line = Pohmelkin
category =  Czech /line = Kacirek
category =  Japanese /line = Enomoto
category =  Vietnamese /line = Thao
category =  Japanese /line = Hasekura
category =  Chinese /line = Rong
category =  Irish /line = John
